In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import IPython.display as ipd

import os
import json
import torch

import commons
import utils
from models import SynthesizerTrn
from text import symbols, accent_symbols, text_to_sequence, accent_to_sequence
from include.preprocess_japanese.synthesize import preprocess_japanese

/home/sarulab/kentaro_seki/.pyenv/versions/3.10.10/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## MB-iSTFT-VITS

In [2]:
output_dir = "output/JSUT"

In [3]:
hps = utils.get_hparams_from_file(os.path.join(output_dir, "config.json"))

In [4]:
net_g = SynthesizerTrn(
    len(symbols),
    len(accent_symbols),
    hps.data.filter_length // 2 + 1,
    hps.train.segment_size // hps.data.hop_length,
    **hps.model).cuda()
net_g.eval()
utils.load_checkpoint(os.path.join(output_dir, "G_1000000.pth"), net_g, None);

Mutli-stream iSTFT VITS


In [5]:
raw_text = "素晴らしい声の人"

with torch.no_grad():
    text, accent = preprocess_japanese(raw_text)
    text, accent = torch.LongTensor(text_to_sequence(text)), torch.LongTensor(accent_to_sequence(accent))
    text = text.unsqueeze(0).cuda()
    accent = accent.unsqueeze(0).cuda()
    x = (text, accent)
    lengths = torch.LongTensor([len(t) for t in text]).cuda()
    audio = net_g.infer(x, lengths, noise_scale=.667, noise_scale_w=0.8, length_scale=1)[0][0,0].data.cpu().float().numpy()
ipd.display(ipd.Audio(audio, rate=hps.data.sampling_rate, normalize=False))